In [13]:
%matplotlib inline 
from collections import OrderedDict
import numpy as np
import pandas as pd

def iter_prefixes(t, i): 
    "from longest to shortest"

    for j in reversed(range(i+1, len(t) + 1)):
        yield t[i:j]


def ms(t, s, i): 
    """the longest prefix of t[i:] that occurs in s"""
    for prefix in iter_prefixes(t, i): 
        #print s + ": " + prefix, 
        if prefix in s:
            #print "*"
            return len(prefix)
        #print
    return 0


def bwt(s):
    from collections import Counter
    s = s + "#" 
    A = np.array([list(s[i:] + s[:i]) for i in range(len(s))])
    As = np.vstack(sorted(A, key=tuple))
    CF = Counter(As[:, 0]) 
    S = sorted(CF)
    C = [0] * (len(CF) + 1)

    for i in range(len(S)):
        if i == 0:
            continue
        C[i] = C[i-1] + CF[S[i-1]]
    C[i+1] = len(s)
    return "".join(As[:, len(s) - 1]), "".join(S), "-".join(map(str, C)), As

def table1(t, s):
    a = pd.DataFrame(OrderedDict([('i', [-1] + range(len(t))),
                                  ('t_i', [''] + list(t)), 
                                  ('MS', [1] + map(lambda i: ms(t, s, i), range(len(t))))]))
    a['nzeros'] = [0] + (a[1:].MS.values - a[0:len(t)].MS.values + 1).tolist()
    a['ms'] = [''] + map(lambda i: "".join(['0'] * i) + '1', a.nzeros[1:])
    a['runs'] = [-1] + map(lambda s: int(s == '1'), a.ms[1:])
    return a.set_index('i')

def table2(t, s):
    bwt_s, alp, C, sa = bwt(s)

    a = pd.DataFrame(OrderedDict([('i', range(len(s) + 1)), 
                                  ('s_i', list(s + "#")), 
                                  ('BWT', list(bwt_s)),
                                  ('SA', map(lambda i: len(s) - sa[i].tolist().index("#"), range(sa.shape[0]))),
                                  ('suff_SA', map(lambda i: "".join(sa[i][:sa[i].tolist().index("#")+1].tolist()), range(sa.shape[0])))
                                 ]))
    return a.set_index('i')
    

#t = 'ceededdbe'
#s = 'cbeeeddba'
s = 'aabbaabbbaabaabaaababbbaab'
t = 'aabbaabbbaabaabaaababbbaab'
#table2(t, s[::-1]).head()

In [16]:
tt = table1(t, s)
tt['pre_merge_RUNS'] = [-1] + [0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
tt['error'] = map(int, tt.pre_merge_RUNS != tt.runs)
tt#[tt.error == 1]

,t_i,MS,nzeros,ms,runs,pre_merge_RUNS,error
i,,,,,,,
-1,,1,0,,-1,-1,0
0,a,26,26,000000000000000000000000001,0,0,0
1,a,25,0,1,1,1,0
2,b,24,0,1,1,1,0
3,b,23,0,1,1,1,0
4,a,22,0,1,1,1,0
5,a,21,0,1,1,1,0
6,b,20,0,1,1,1,0
7,b,19,0,1,1,1,0


In [12]:
table2(t, s)#[50:80]

,s_i,BWT,SA,suff_SA
i,,,,
0,b,a,8,#
1,e,b,7,a#
2,b,e,5,aba#
3,b,a,6,ba#
4,e,e,2,bbeaba#
5,a,b,3,beaba#
6,b,#,0,bebbeaba#
7,a,b,4,eaba#
8,#,b,1,ebbeaba#
